# Synthesis of Toroidal Hyper-Helices

## Algebraically by Hand

Assume iteration $0$ is a point a the origin.

Here are the steps of transformation for each iteration:

1. Scale the time by $f_T$ so that the curve "wraps" $f_T$ times.
2. Rotate the whole curve by $\frac{\pi}{2}$ about the $x$ axis.
3. Scale the whole curve down by $f_R$.
4. Translate the whole curve by $1$ along the $x$ axis.
5. Unwind the curve about the unit cicle in the $xy$ plane by applying a rotation about the $z$ axis by the parameter $t$.

Putting this all together we get the recurrence relation:

$$ \vec{H}_{n}(t) = R_{z}(2 \pi t) \left( \frac{1}{f_{R}} R_{x}\left(\frac{\pi}{2}\right) \vec{H}_{n-1}(f_{T} t) + \hat{i}_{x} \right) $$

Or more explicitly:

$$ \vec{H}_{n}(t) = 
\left[\begin{matrix}
    \cos{\left(2 \pi t \right)} & \sin{\left(2 \pi t \right)} & 0\\
    - \sin{\left(2 \pi t \right)} & \cos{\left(2 \pi t \right)} & 0\\
    0 & 0 & 1
    \end{matrix}\right]
\left( 
    \left[\begin{matrix}
        1 & 0 & 0\\
        0 & 0 & 1\\
        0 & -1 & 0
    \end{matrix}\right] 
    \vec{H}_{n-1}(f_{T} t) + 
    \left[\begin{matrix}
        1\\
        0\\
        0
    \end{matrix}\right]
\right) $$
$$ \vec{H}_{n}(t) = 
\left[\begin{matrix}
    \cos{\left(2 \pi t \right)} & 0 & \sin{\left(2 \pi t \right)}\\
    - \sin{\left(2 \pi t \right)} & 0 & \cos{\left(2 \pi t \right)}\\
    0 & -1 & 0
\end{matrix}\right]
\vec{H}_{n-1}(f_{T} t) + 
\left[\begin{matrix}
    \cos{\left(2 \pi t \right)}\\
    - \sin{\left(2 \pi t \right)}\\
    0
\end{matrix}\right]
$$

Where
- $\vec{H}_{n}(t)$ is the parametric equation expression for the hyperhelix of iteration $n$.
- $t$ is the parameter of the parametrized expressions, which has range $[0,1]$.
- $R_{x}(t)$ is the 3D rotation matrix about the $x$ axis, by $t$ radians.
- $\hat{i}_{x}$ is the unit vector in the $x$ direction.
- $f_{R}$ is the scaling ratio of the hyper helix size/radius between iterations.
- $f_{T}$ is the scaling ratio of the hyper helix frequency between iterations.

Notice this is a linear recurrence relation, and so we can easily derive a closed form expression in terms of repeated sum of a repeated product of matrices, all applied to the unit vector. Could we simplify that? I'm not convinced since the changing frequencies mean it's not as simple as our usual matrix powers which can be simplified via diagonalization. But maybe by noticing we can express each rotation matrix in terms of the smallest rotation matrix to the power of their frequency ratios, maybe we could find something? For another time perhaps!

## Algebraically via Sympy

### Synthesis

In [ ]:
import sympy

In [ ]:
f_T, f_R = sympy.symbols(['f_T','f_R'])

In [ ]:
t, = sympy.symbols(['t'])

In [ ]:
redir_mat = sympy.rot_axis1(2*sympy.pi/4)
offset = sympy.Matrix([1,0,0])
def hyperhelix(i, t):
    if i == 0:
        return sympy.Matrix([0,0,0])
    return sympy.rot_axis3(2*sympy.pi * t) * (redir_mat * hyperhelix(i-1, f_T * t) / f_R + offset)

### Examples

In [ ]:
hyperhelix(1, t)

In [ ]:
hyperhelix(2, t)

In [ ]:
hyperhelix(3, t)

### Graphing

In [ ]:
from sympy.plotting.plot import MatplotlibBackend

In [ ]:
# hack around bug in sympy's plotting
class EqualAxesBackend(MatplotlibBackend):
    def process_series(self):
        super().process_series()
        for ax in self.ax:
            ax.set(xlim=(-1, 1), ylim=(-1, 1), zlim=(-1, 1))

In [ ]:
iterations = 3
R_factor = 4
T_factor = 16
resolution_factor = 16+1

In [ ]:
resolution = resolution_factor * T_factor ** (iterations-1)

In [ ]:
p = hyperhelix(iterations, t)
p

In [ ]:
ax = sympy.plotting.plot3d_parametric_line(*tuple(p.subs({f_R:R_factor, f_T:T_factor})), (t, 0, 1), nb_of_points=resolution, backend=EqualAxesBackend)

## Numerically via Numpy

### Synthesis

In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation as R

In [ ]:
redir_mat = R.from_rotvec(2 * np.pi / 4 * np.array([1, 0, 0])).as_matrix()
def hyperhelix_np(iterations, T_factor, R_factor, resolution_factor=16+1):
    resolution = resolution_factor * T_factor ** (iterations-1)
    t = np.linspace(0,1,resolution)
    p = np.zeros((resolution,3,1))
    for i in reversed(range(iterations)):
        p = np.matmul(redir_mat,p)
        p += np.array([[1,0,0]]).T / R_factor**i        
        rot_mats = R.from_rotvec(2 * np.pi * T_factor**i * np.outer(t, np.array([0, 0, 1]))).as_matrix()
        p = np.matmul(rot_mats,p)
    return np.squeeze(p)

### Graphing

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
iterations = 3
R_factor = 4
T_factor = 16
resolution_factor = 16+1

In [ ]:
p = hyperhelix_np(iterations, T_factor, R_factor, resolution_factor)
p.shape

In [ ]:
ax = plt.figure().add_subplot(projection='3d')
ax.set(xlim=(-1, 1), ylim=(-1, 1), zlim=(-1, 1))
ax.plot(*p.T)

plt.show()